In [2]:
import pandas as pd
import os

In [4]:
files_dir = "world_bank_projects"
files_list = os.listdir(files_dir)

In [60]:
tables = []
for f in files_list:
    if f[-4:] == 'xlsx':
        table = pd.read_excel(os.path.join(files_dir,f),skiprows=1)
        tables.append(table)

In [61]:
tables[0].head()

,Project Id,Region,Country,Project Status,Project Name,Project Development Objective,Implementing Agency,Consultant Services Required,Project URL,Board Approval Date,...,Grant Amount,Borrower,Lending Instrument,Environmental Assessment Category,Environmental and Social Risk,Sector 1,Sector 2,Sector 3,Theme 1,Theme 2
0,P004092,East Asia and Pacific,Republic of Korea,Closed,Gojeong Power Project,NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,NaN,...,0,NaN,Specific Investment Loan,NaN,NaN,(Historic)Thermal,NaN,NaN,NaN,NaN
1,P004233,East Asia and Pacific,Malaysia,Closed,Population Project,NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,NaN,...,0,NaN,Sector Investment and Maintenance Loan,NaN,NaN,"(Historic)Primary health, including reproducti...",NaN,NaN,NaN,NaN
2,P009174,Europe and Central Asia,"Macedonia, Republic of North",Closed,Agriculture & Agroindustry 2 Project (Macedonia),NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,1980-02-01T00:00:00Z,...,0,NaN,Specific Investment Loan,C,NaN,(Historic)Agricultural credit,NaN,NaN,NaN,NaN
3,P007361,Latin America and Caribbean,Republic of Honduras,Closed,Industrial Credit Project (02),NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,1981-12-22T00:00:00Z,...,0,NaN,Financial Intermediary Loan,NaN,NaN,(Historic)Financial sector development,NaN,NaN,NaN,NaN
4,P003277,Africa East,Republic of Zimbabwe,Closed,Railway Development Project,NaN,NaN,NaN,https://projects.worldbank.org/en/projects-ope...,1983-08-02T00:00:00Z,...,0,NaN,Technical Assistance Loan,NaN,NaN,Railways,NaN,NaN,NaN,NaN


In [62]:
table = pd.concat(tables,ignore_index=True)

In [64]:
table['IBRD Commitment'] = table['IBRD Commitment'].apply(lambda x: x.replace(',',''))
table['IBRD Commitment'] = table['IBRD Commitment'].astype(float)

In [65]:
table['IDA Commitment'] = table['IDA Commitment'].apply(lambda x: x.replace(',',''))
table['IDA Commitment'] = table['IDA Commitment'].astype(float)

In [66]:
table['Total IDA and IBRD Commitment'] = table['Total IDA and IBRD Commitment'].apply(lambda x: x.replace(',',''))
table['Total IDA and IBRD Commitment'] = table['Total IDA and IBRD Commitment'].astype(float)

In [57]:
table.to_csv(os.path.join(files_dir,"world_bank_projects.csv"),index=False)


It turns out that the Excel files are for some reason not formed correctly by the World Bank site. Instead, we use the JSON-based API: 
https://search.worldbank.org/api/v2/projects
  ?format=json
  &fl=id,regionname,countryname,projectstatusdisplay,project_name,countryshortname,pdo,
  impagency,cons_serv_reqd_ind,url,boardapprovaldate,closingdate,projectfinancialtype,
  curr_project_cost,ibrdcommamt,idacommamt,totalamt,grantamt,borrower,lendinginstr,
  envassesmentcategorycode,esrc_ovrl_risk_rate,sector1,sector2,sector3,
  theme1,theme2,%20%20status,totalcommamt,proj_last_upd_date
  &apilang=en
  &projectfinancialtype_exact=IBRD
  &rows=4000&os=0

In [5]:
wb_json = pd.read_json(os.path.join(files_dir,'wbapiall_clean.json'),orient='index')

In [6]:
wb_json.describe()

,id,regionname,countryname,lendinginstr,projectstatusdisplay,status,project_name,boardapprovaldate,ibrdcommamt,idacommamt,...,theme1,url,totalcommamt,cons_serv_reqd_ind,closingdate,envassesmentcategorycode,sector2,sector3,project_abstract,theme2
count,3697,3697,3697,3678,3697,3697,3697,3685,3697,3697,...,3697,3697,3697,190,3424,2976,2943,1994,3252,2783
unique,3697,8,106,16,2,2,3526,1872,966,129,...,886,3697,976,3,640,7,1276,875,3194,912
top,P168630,Latin America and Caribbean,[People's Republic of China],Specific Investment Loan,Closed,Closed,Urban Transport Project,2001-06-26T00:00:00Z,"100,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"100,000,000",Y,2021-12-31T00:00:00Z,B,{'Name': 'Central Government (Central Agencies...,"{'Name': 'Sub-National Government', 'Percent': 5}",{'cdata': 'This report combines five emergency...,{'Name': 'Infrastructure services for private ...
freq,1,1248,316,1475,2953,2953,6,10,242,3461,...,496,1,242,104,74,1373,31,22,5,32


In [7]:
wb_json.head()

,id,regionname,countryname,lendinginstr,projectstatusdisplay,status,project_name,boardapprovaldate,ibrdcommamt,idacommamt,...,theme1,url,totalcommamt,cons_serv_reqd_ind,closingdate,envassesmentcategorycode,sector2,sector3,project_abstract,theme2
P172024,P172024,Europe and Central Asia,[Republic of Croatia],Investment Project Financing,Active,Active,Helping Enterprises Access Liquidity in the Re...,2021-05-14T00:00:00Z,"242,100,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"242,100,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
P173975,P173975,Europe and Central Asia,[Georgia],Investment Project Financing,Active,Active,"Georgia Relief and Recovery for Micro, Small, ...",2021-05-11T00:00:00Z,"102,900,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"102,900,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
P175895,P175895,Europe and Central Asia,[Ukraine],Investment Project Financing,Active,Active,Ukraine Emergency COVID-19 Response and Vaccin...,2021-05-10T00:00:00Z,"90,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"90,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
P171050,P171050,Europe and Central Asia,[Ukraine],Investment Project Financing,Active,Active,Ukraine Improving Higher Education for Results...,2021-05-05T00:00:00Z,"200,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"200,000,000",N,NaN,NaN,NaN,NaN,NaN,NaN
P175138,P175138,Latin America and Caribbean,[Argentine Republic],Investment Project Financing,Active,Active,Buenos Aires &#8211; Mitre Passenger Railway L...,2021-04-30T00:00:00Z,"347,000,000",0,...,"{'Name': '', 'Percent': 0}",https://projects.worldbank.org/en/projects-ope...,"347,000,000",Y,2026-06-30T00:00:00Z,NaN,NaN,NaN,NaN,NaN


In [8]:
wb_json.ibrdcommamt = wb_json.ibrdcommamt.apply(lambda x: x.replace(',',''))
wb_json.ibrdcommamt = wb_json.ibrdcommamt.astype(float)

In [9]:
import numpy as np

In [10]:
wb_json['sector1_name'] = [x['Name'] for x in wb_json['sector1']]
wb_json['sector1_pct'] = [x['Percent'] for x in wb_json['sector1']]
wb_json['sector2_name'] = [x['Name'] if type(x) == dict else np.nan for x in wb_json['sector2']]
wb_json['sector2_pct'] = [x['Percent'] if type(x) == dict else np.nan for x in wb_json['sector2']]
wb_json['sector3_name'] = [x['Name'] if type(x) == dict else np.nan for x in wb_json['sector3']]
wb_json['sector3_pct'] = [x['Percent'] if type(x) == dict else np.nan for x in wb_json['sector3']]

In [11]:
import pickle

In [12]:
wb_json.boardapprovaldate = wb_json.boardapprovaldate.apply(pd.Timestamp)

In [13]:
wb_json.boardapprovaldate.describe()

<ipython-input-13-c98746017877>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  wb_json.boardapprovaldate.describe()


count                          3685
unique                         1872
top       2001-06-26 00:00:00+00:00
freq                             10
first     1980-02-01 00:00:00+00:00
last      2021-05-14 00:00:00+00:00
Name: boardapprovaldate, dtype: object

In [16]:
wb_json.sector1_name.describe()

count                                      3697
unique                                       83
top       Central Government (Central Agencies)
freq                                        456
Name: sector1_name, dtype: object

In [33]:
wb_json.sector1_name.value_counts().head(40)

Central Government (Central Agencies)                             456
Health                                                            189
Social Protection                                                 188
Rural and Inter-Urban Roads                                       187
Other Industry, Trade and Services                                175
Banking Institutions                                              150
Sub-National Government                                           131
Water Supply                                                      125
Other Water Supply, Sanitation and Waste Management               122
Roads and highways                                                120
Irrigation and Drainage                                           110
Other Energy and Extractives                                      107
Other Public Administration                                        97
Power                                                              96
Other Agriculture, F

In [18]:
wb_json.sector1_name.describe()

count                                      3697
unique                                       83
top       Central Government (Central Agencies)
freq                                        456
Name: sector1_name, dtype: object

In [24]:
with open("world_bank_sectors.txt") as f:
  sectors = f.read()

sectors_list = sectors.split("\n")

sector_dict = dict()
current_major_sector = ''
for sector in sectors_list:
    if sector == '':
        current_major_sector = ''
        continue
    if current_major_sector == '':
        current_major_sector = sector
        sector_dict[current_major_sector] = []
        continue
    if current_major_sector in sector_dict.keys():
        sector_dict[current_major_sector].append(sector)

reverse_sector_dict = dict()
for major_sector in sector_dict:
    for sector in sector_dict[major_sector]:
        reverse_sector_dict[sector] = major_sector

In [26]:
wb_json.sector1_name.apply(lambda x: x in reverse_sector_dict).value_counts()

True     3328
False     369
Name: sector1_name, dtype: int64

In [34]:
wb_json['major_sector_name'] = [reverse_sector_dict[minor_sector] if minor_sector in reverse_sector_dict else None for minor_sector in wb_json.sector1_name.values]

In [35]:
wb_json.major_sector_name.describe()

count             3328
unique              11
top       Public Admin
freq               716
Name: major_sector_name, dtype: object

In [36]:
wb_json.to_pickle(os.path.join(files_dir,"world_bank_projects_json.pkl"))

In [40]:
wb_json.tail(10)

,id,regionname,countryname,lendinginstr,projectstatusdisplay,status,project_name,boardapprovaldate,ibrdcommamt,idacommamt,...,sector3,project_abstract,theme2,sector1_name,sector1_pct,sector2_name,sector2_pct,sector3_name,sector3_pct,major_sector_name
P004891,P004891,Middle East and North Africa,[People's Democratic Republic of Algeria],Specific Investment Loan,Closed,Closed,Highway Project (03),NaT,126000000.0,0,...,NaN,{'cdata': 'The project will help relieve the m...,NaN,(Historic)Highways,100,NaN,NaN,NaN,NaN,None
P004995,P004995,Middle East and North Africa,[Arab Republic of Egypt],Specific Investment Loan,Closed,Closed,Suez Canal Rehabilitation Project,NaT,50000000.0,0,...,NaN,{'cdata': 'The Suez Canal Rehabilitation Proje...,NaN,Ports/Waterways,100,NaN,NaN,NaN,NaN,Transportation
P005387,P005387,Middle East and North Africa,[Kingdom of Morocco],Specific Investment Loan,Closed,Closed,Phosphate Fertilizer Expansion Project,NaT,50000000.0,0,...,NaN,NaN,NaN,(Historic)Mining and other extractive,100,NaN,NaN,NaN,NaN,None
P005547,P005547,Middle East and North Africa,[Syrian Arab Republic],Sector Investment and Maintenance Loan,Closed,Closed,Mehardeh Power Project,NaT,25000000.0,0,...,NaN,{'cdata': 'The Mehardeh Power Project includes...,NaN,(Historic)Thermal,100,NaN,NaN,NaN,NaN,None
P008346,P008346,Europe and Central Asia,[Republic of Cyprus],Specific Investment Loan,Closed,Closed,Paphos Irrigation Project,NaT,14000000.0,0,...,NaN,{'cdata': 'The project would construct three w...,NaN,Irrigation and Drainage,100,NaN,NaN,NaN,NaN,Agriculture
P003748,P003748,East Asia and Pacific,[Republic of Indonesia],Specific Investment Loan,Closed,Closed,Irrigation Project (07),NaT,33000000.0,0,...,NaN,{'cdata': 'Major components of the project are...,NaN,Irrigation and Drainage,100,NaN,NaN,NaN,NaN,Agriculture
P003771,P003771,East Asia and Pacific,[Republic of Indonesia],Specific Investment Loan,Closed,Closed,Nucleus Estates Project (02) Smallholders Proj...,NaT,65000000.0,0,...,NaN,{'cdata': 'In its settlement component the Sec...,NaN,(Historic)Perennial crops,100,NaN,NaN,NaN,NaN,None
P004092,P004092,East Asia and Pacific,[Republic of Korea],Specific Investment Loan,Closed,Closed,Gojeong Power Project,NaT,115000000.0,0,...,NaN,{'cdata': 'The project constitutes the next lo...,NaN,(Historic)Thermal,100,NaN,NaN,NaN,NaN,None
P004233,P004233,East Asia and Pacific,[Malaysia],Sector Investment and Maintenance Loan,Closed,Closed,Population Project,NaT,5000000.0,0,...,NaN,{'cdata': 'The project consists of increments ...,NaN,"(Historic)Primary health, including reproducti...",100,NaN,NaN,NaN,NaN,None
P008921,P008921,Europe and Central Asia,[Republic of Turkey],Specific Investment Loan,Closed,Closed,Grain Storage Project,NaT,85000000.0,0,...,NaN,{'cdata': 'The main objectives of the proposed...,NaN,(Historic)Agro-industry and marketing,100,NaN,NaN,NaN,NaN,None
